# Reinforcement learning and the provision of public goods 


The following code is the base for the Bachelor thesis:"Reinforcement learning and the provision of public goods - An evaluation of a reinforcement learning model in large public goods games"
The single steps are all commented and code will be cut into many pieces to improve intelligibility. The code includes different optimization options and additional tools to maximize usability.

In [ ]:
# Save Multiple Data for Plotting
DataToSave1 = []
DataToSave2 = []

In [ ]:
# General Imports
import numpy
import matplotlib.pyplot as plt
import random
import scipy, scipy.stats
import pylab


In [ ]:
# General Parameters
numberOfPlayers = 100
moneyToDistribute = 50
roundsToPlay = 100
MPCR = 0.3

multiplierPublicGood = MPCR*numberOfPlayers
numberOfMCSimulations = 10

# Model 1: Reinforcement learning

In [ ]:
# 1. Parameters Reinforcement learning model

cutoff = 0.000
experimentation = 0.05
forgetting = 0.95
initialPropensity = 700.00

In [ ]:
# 2. General Data // General objects/ arrays are initialized

initialPropensities = []
possibleDecisions = []

for x in range (0,moneyToDistribute+1): 
   initialPropensities.append(initialPropensity)
  
for x in range (0,moneyToDistribute+1):
    possibleDecisions.append(x)


In [ ]:
# 3. Initial agent values // an object with all propensity stacks is created

propensityStack = numpy.array([initialPropensities,initialPropensities])
propensitiesToAdd = numpy.array([initialPropensities])
for x in range (0,(numberOfPlayers-2)):
    propensityStack = numpy.concatenate((propensityStack,propensitiesToAdd),axis=0)
print (propensityStack)



In [ ]:
# 4.  Converting data to %

def convert_to_percent (values,moneyToDistribute,roundsToPlay):
    converted=[]
    for x in range (0,roundsToPlay):
        converted.append((values[x] /moneyToDistribute)*100)
    return(converted)

In [ ]:
# 5. Reset the propensity stacks after every simulation

def reset_propensity(initialPropensities,numberOfPlayers):
    propensityStack = numpy.array([initialPropensities,initialPropensities])
    propensitiesToAdd = numpy.array([initialPropensities])

    for x in range (0,(numberOfPlayers-2)):
       propensityStack = numpy.concatenate((propensityStack,propensitiesToAdd),axis=0)

    return (propensityStack)

In [ ]:
# 6. Decide play of agents  // Based on the propensity stack, probabilitys for all player are calculated and a play
# is drawn from the probability distribution.

def decide_play(propensityStack,possibleDecisions) :
    playsThisRound = []
    for x in range (0,numpy.size(propensityStack,0)) :   
        myProbabilities = propensityStack[x]/numpy.sum(propensityStack[x])   
        myPlay = numpy.random.choice(possibleDecisions,p=myProbabilities)
        playsThisRound.append(myPlay)
    print(propensityStack[0])
    return playsThisRound

In [ ]:
# 7. The game  //  The public good is calculated and based on that also the return for every player

def play_game(multiplierPublicGood,numberOfPlayers,playsCurrentRound):
    ROI = (numpy.sum(playsCurrentRound)*multiplierPublicGood)/numberOfPlayers
    return ROI


In [ ]:
# 8. Updating agents propensities // The Difference between contribution and ROI is calculated and added to 
# the propensity stack of the chosen play for all players
# cutoff and experimentation are calculated and added to the propensity stack

def update_propensities(numberOfPlayers,propensityStack,playsCurrentRound,ROI,experimentation,possibleDecisions,cutoff, multiplierPublicGood):
    for x in range (0,numberOfPlayers) :
      difference = ROI - playsCurrentRound[x]
      if(difference>=0):
          propensityStack[x,playsCurrentRound[x]] =propensityStack[x,playsCurrentRound[x]]+(difference)*(1-experimentation)
      else : propensityStack[x,playsCurrentRound[x]] =propensityStack[x,playsCurrentRound[x]]+(difference)*(1)
      preProbabilities = propensityStack[x]/numpy.sum(propensityStack[x])
      for y in range (0,len(possibleDecisions)) :
        if preProbabilities[y] < cutoff:
           propensityStack[x,y] = 0 
      if  (playsCurrentRound[x] == len(possibleDecisions)-1 and difference >= 0) :
        propensityStack[x,playsCurrentRound[x]-1]= propensityStack[x,playsCurrentRound[x]-1]+(difference)*(experimentation/2)
        propensityStack[x,playsCurrentRound[x]]= propensityStack[x,playsCurrentRound[x]]+(difference)*(experimentation/2)
      elif playsCurrentRound[x] == 0 and difference >= 0:
        propensityStack[x,playsCurrentRound[x]+1]= propensityStack[x,playsCurrentRound[x]+1]+(difference)*(experimentation/2)
        propensityStack[x,playsCurrentRound[x]]= propensityStack[x,playsCurrentRound[x]]+(difference)*(experimentation/2)
      elif difference >= 0:
        propensityStack[x,playsCurrentRound[x]+1]= propensityStack[x,playsCurrentRound[x]+1]+(difference)*(experimentation/2)
        propensityStack[x,playsCurrentRound[x]-1]= propensityStack[x,playsCurrentRound[x]-1]+(difference)*(experimentation/2)
    return propensityStack

    

In [ ]:
# 9. Forgetting propensities // Propensities are altered bases on the forgetting parameter

def forget_propensities(numberOfPlayers,propensityStack,forgetting):
    for x in range (0,numberOfPlayers) :
        propensityStack[x] = propensityStack[x]*forgetting
    return(propensityStack)

In [ ]:
# 10. Play the complete game // The functions of the model are executed one after another for the number of rounds that 
# were specified. After that, the plays of all players during all rounds are stored in an object

def complete_simulation(roundsToPlay,decide_play,propensityStack,possibleDecisions,play_game,multiplierPublicGood,numberOfPlayers,update_propensities,experimentation,cutoff,forget_propensities,forgetting):
    playsAllGame = []
    playsSpecificPlayer = []
    for x in range (0,roundsToPlay):
        playsThisRound = decide_play(propensityStack,possibleDecisions)
        playsAllGame.append(playsThisRound)
        playsSpecificPlayer.append(playsThisRound[0])
        gameResult = play_game(multiplierPublicGood,numberOfPlayers,playsThisRound)
        propensityStack = update_propensities(numberOfPlayers,propensityStack,playsThisRound,gameResult,experimentation,possibleDecisions,cutoff,multiplierPublicGood)
        propensityStack = forget_propensities(numberOfPlayers,propensityStack,forgetting)
    print(playsSpecificPlayer)
    return(playsAllGame)

In [ ]:
# 11. Initiate a single simulation and prepare the data for plotting // The data object is processed into an array that holds 
# the average contribution for every round, and in arrays that hold the individual contributions for every round

singleSimulation =complete_simulation(roundsToPlay,decide_play,propensityStack,possibleDecisions,play_game,multiplierPublicGood,numberOfPlayers,update_propensities,experimentation,cutoff,forget_propensities,forgetting)
propensityStack = reset_propensity(initialPropensities,numberOfPlayers)
averageContribution = []
specificContribution =[]
freeRiding = []
for x in range (0,roundsToPlay):
    averageContribution.append((sum(singleSimulation[x])/numberOfPlayers))

for z in range (0,roundsToPlay): 
    frCounter = 0
    for w in range(0, numberOfPlayers) :
        if singleSimulation[z][w] == 0 : 
            frCounter +=1
    frCounter = frCounter/numberOfPlayers*100
    freeRiding.append(frCounter)

for y in range (0,numberOfPlayers):
    playersContribution = []
    for z in range(0,roundsToPlay): 
        playersContribution.append(singleSimulation[z][y])
    specificContribution.append(playersContribution)


In [ ]:
# 12. Plotting the decisions for a single simulation  // The average contribution for every round is shown as a graph.
plt.plot(convert_to_percent(averageContribution,moneyToDistribute,roundsToPlay))
plt.ylabel('AverageContribution in %')
plt.show()

In [ ]:
# 13. Plotting the free-riding behavior for a single simulation // 
# The % of free riding behavior for every round is shown as a graph
print(freeRiding)
plt.plot(freeRiding)
plt.ylabel('Free riding behavior in %')
plt.show()

In [ ]:
# 14. Plotting the decisions for a single simulation  // Individual contributions for every round are drawn as a graph
for x in range (0,numberOfPlayers): 
    plt.plot(convert_to_percent(specificContribution[x],moneyToDistribute,roundsToPlay))
    plt.ylabel("contribution of player %x in percent" % (x+1))
    '''plt.show()'''

In [ ]:
# 15. Average Contribution in Numbers for a single simulation
print(averageContribution)

In [ ]:
# 16. Initiate a monte carlo simulation of the game // The data is processed into an array that holds 
# the average contribution for every round for all simulations
mcContainerAverage  = []
mcCOntainerFreeRiding = []
for x in range (0,numberOfMCSimulations):
    singleSimulation =complete_simulation(roundsToPlay,decide_play,propensityStack,possibleDecisions,play_game,multiplierPublicGood,numberOfPlayers,update_propensities,experimentation,cutoff,forget_propensities,forgetting)
    propensityStack = reset_propensity(initialPropensities,numberOfPlayers)
    averageContributionX = []
    freeRidingX = []
    for x in range (0,roundsToPlay):
        averageContributionX.append((sum(singleSimulation[x])/numberOfPlayers))
    mcContainerAverage.append(averageContributionX)
    
    for z in range (0,roundsToPlay): 
        frCounter = 0
        for w in range(0, numberOfPlayers) :
            if singleSimulation[z][w] == 0 : 
                frCounter +=1
        frCounter = frCounter/numberOfPlayers*100
        freeRidingX.append(frCounter)
    mcCOntainerFreeRiding.append(freeRidingX)
    
    

print(mcContainerAverage,mcCOntainerFreeRiding)


In [ ]:
# 17. Plotting the decisions for all simulations  // The average contribution for every round for all simulations is shown as a graph.

for x in range(0,numberOfMCSimulations):
    plt.plot(convert_to_percent(mcContainerAverage[x],moneyToDistribute,roundsToPlay))
plt.ylabel('AverageContribution in %')
plt.show()

In [ ]:
# 18. Plotting the free riding behavior for all simulations  // Free riding for every round for all simulations is shown as a graph.
for x in range(0,numberOfMCSimulations):
   
    plt.plot(mcCOntainerFreeRiding)
plt.ylabel('Average FreeRiding in %')
plt.show()

In [ ]:
# 19. Save specific results 60/0.02 : 100/0.02 : 60/0.04 : 100/0.04
WeimannA.append(averageContribution)
WeimannF.append(freeRiding)


In [ ]:
# 20. Print specific data
print(WeimannA)
for x in range(0,4):
    plt.plot(convert_to_percent(WeimannA[x],moneyToDistribute,roundsToPlay))

plt.ylabel('Average contribution in %')
plt.xlabel('blue=40/0.3  yellow=40/0.75  green= 100/0.3  red = 100/0.75')
plt.show()

In [ ]:
# 21. Print specific data
print(WeimannF)
for x in range(0,4):
    plt.plot(WeimannF[x])
plt.ylabel('Average FreeRiding in %')
plt.xlabel('blue=60/0.02  yellow=100/0.02  green= 60/0.04  red = 100/0.04')
plt.show()

In [ ]:
# 22. Export results
f= open("IsaacInitial.txt","a+") # put new datafile names here
f.write(str(WeimannA))
f.write(str(WeimannF))

-----------------------------------------------------------------------------------------------------------------------

In [ ]:
# 23.  Fitting the parameters // Given a number of players, rounds to play, MPCR, and Money to distribute, the parameters
# cutoff, experimentation, forgetting ,initialPropensity are fitted in order to maximize the predictive power of the model.
# Unfortunately this can only be done by a graphical analysis. Graphs for all combinations of parameters are printed below.
# !Keep the parameters which seems to create a good prediction. Only rounded parameter values are tested.

propensityLimit = 40
for w in range (1,propensityLimit+1) :
    initialPropensity = w*2.0
    initialPropensities = []
    for x in range (0,moneyToDistribute+1): 
        initialPropensities.append(initialPropensity)
    mcContainerAverage  = []
    for z in range (0,numberOfMCSimulations):
        propensityStack = reset_propensity(initialPropensities,numberOfPlayers)
        singleSimulation =complete_simulation(roundsToPlay,decide_play,propensityStack,possibleDecisions,play_game,multiplierPublicGood,numberOfPlayers,update_propensities,experimentation,cutoff,forget_propensities,forgetting)
        averageContribution = []
        for x in range (0,roundsToPlay):
            averageContribution.append((sum(singleSimulation[x])/numberOfPlayers))
        mcContainerAverage.append(averageContribution)
            
    print(initialPropensity)
                          
    for y in range(0,numberOfMCSimulations):
        plt.plot(convert_to_percent(mcContainerAverage[y],moneyToDistribute,roundsToPlay))
    plt.ylabel('AverageContribution in %')
    plt.show()
                


In [ ]:
# 24. Fitting experimentation parameter: 

experimentationLimit = 25
for x in range (0,experimentationLimit+1) :
    experimentation = x/50
    mcContainerAverage  = []
    mcCOntainerFreeRiding = []
    for x in range (0,numberOfMCSimulations):
        singleSimulation =complete_simulation(roundsToPlay,decide_play,propensityStack,possibleDecisions,play_game,multiplierPublicGood,numberOfPlayers,update_propensities,experimentation,cutoff,forget_propensities,forgetting)
        propensityStack = reset_propensity(initialPropensities,numberOfPlayers)
        averageContributionX = []
        freeRidingX = []
        
        for x in range (0,roundsToPlay):
            averageContributionX.append((sum(singleSimulation[x])/numberOfPlayers))
        mcContainerAverage.append(averageContributionX)

        for z in range (0,roundsToPlay): 
            frCounter = 0
            for w in range(0, numberOfPlayers) :
                if singleSimulation[z][w] == 0 : 
                    frCounter +=1
            frCounter = frCounter/numberOfPlayers*100
            freeRidingX.append(frCounter)
        mcCOntainerFreeRiding.append(freeRidingX)
    print(experimentation)    
    for x in range(0,numberOfMCSimulations):
        plt.plot(convert_to_percent(mcContainerAverage[x],moneyToDistribute,roundsToPlay))
    plt.ylabel('AverageContribution in %')
    plt.show()

In [ ]:
# 25. Fitting forgetting parameter: 

forgettingLimit = 24
for x in range (0,forgettingLimit+1) :
    forgetting = 1-(x/50)
    mcContainerAverage  = []
    mcCOntainerFreeRiding = []
    for x in range (0,numberOfMCSimulations):
        singleSimulation =complete_simulation(roundsToPlay,decide_play,propensityStack,possibleDecisions,play_game,multiplierPublicGood,numberOfPlayers,update_propensities,experimentation,cutoff,forget_propensities,forgetting)
        propensityStack = reset_propensity(initialPropensities,numberOfPlayers)
        averageContributionX = []
        freeRidingX = []
        
        for x in range (0,roundsToPlay):
            averageContributionX.append((sum(singleSimulation[x])/numberOfPlayers))
        mcContainerAverage.append(averageContributionX)

        for z in range (0,roundsToPlay): 
            frCounter = 0
            for w in range(0, numberOfPlayers) :
                if singleSimulation[z][w] == 0 : 
                    frCounter +=1
            frCounter = frCounter/numberOfPlayers*100
            freeRidingX.append(frCounter)
        mcCOntainerFreeRiding.append(freeRidingX)
    print(forgetting)    
    for x in range(0,numberOfMCSimulations):
        plt.plot(convert_to_percent(mcContainerAverage[x],moneyToDistribute,roundsToPlay))
    plt.ylabel('AverageContribution in %')
    plt.show()

In [ ]:
# 26. Fitting cutoff parameter: 

cutoffLimit = 50
for x in range (0,cutoffLimit+1) :
    cutoff = x/1000
    mcContainerAverage  = []
    mcCOntainerFreeRiding = []
    for x in range (0,numberOfMCSimulations):
        singleSimulation =complete_simulation(roundsToPlay,decide_play,propensityStack,possibleDecisions,play_game,multiplierPublicGood,numberOfPlayers,update_propensities,experimentation,cutoff,forget_propensities,forgetting)
        propensityStack = reset_propensity(initialPropensities,numberOfPlayers)
        averageContributionX = []
        freeRidingX = []
        
        for x in range (0,roundsToPlay):
            averageContributionX.append((sum(singleSimulation[x])/numberOfPlayers))
        mcContainerAverage.append(averageContributionX)

        for z in range (0,roundsToPlay): 
            frCounter = 0
            for w in range(0, numberOfPlayers) :
                if singleSimulation[z][w] == 0 : 
                    frCounter +=1
            frCounter = frCounter/numberOfPlayers*100
            freeRidingX.append(frCounter)
        mcCOntainerFreeRiding.append(freeRidingX)
    print(cutoff)    
    for x in range(0,numberOfMCSimulations):
        plt.plot(convert_to_percent(mcContainerAverage[x],moneyToDistribute,roundsToPlay))
    plt.ylabel('AverageContribution in %')
    plt.show()

In [ ]:
# 27. Simulate the final model with specific parameters 
experimentation = 0.05
forgetting = 0.96
initialPropensity = 3.0
initialPropensities = []

for x in range (0,moneyToDistribute+1): 
        initialPropensities.append(initialPropensity*1.0)

propensityStack = numpy.array([initialPropensities,initialPropensities])
propensitiesToAdd = numpy.array([initialPropensities])
for x in range (0,(numberOfPlayers-2)):
    propensityStack = numpy.concatenate((propensityStack,propensitiesToAdd),axis=0)
    
    
singleSimulation =complete_simulation(roundsToPlay,decide_play,propensityStack,possibleDecisions,play_game,multiplierPublicGood,numberOfPlayers,update_propensities,experimentation,cutoff,forget_propensities,forgetting)
propensityStack = reset_propensity(initialPropensities,numberOfPlayers)
averageContribution = []
specificContribution =[]
freeRiding = []
for x in range (0,roundsToPlay):
    averageContribution.append((sum(singleSimulation[x])/numberOfPlayers))

for z in range (0,roundsToPlay): 
    frCounter = 0
    for w in range(0, numberOfPlayers) :
        if singleSimulation[z][w] == 0 : 
            frCounter +=1
    frCounter = frCounter/numberOfPlayers*100
    freeRiding.append(frCounter)

for y in range (0,numberOfPlayers):
    playersContribution = []
    for z in range(0,roundsToPlay): 
        playersContribution.append(singleSimulation[z][y])
    specificContribution.append(playersContribution)

plt.plot(convert_to_percent(averageContribution,moneyToDistribute,roundsToPlay))
plt.ylabel('AverageContribution in %')
plt.show()

plt.plot(freeRiding)
plt.ylabel('Free riding in every round')
plt.show()

----------------------- The end ---------------------------------------------------------------------------------------